In [1]:
#保養品與評論者相關資訊(無心得)

import shutil
import os
import time
import sys
import requests
import re
from bs4 import BeautifulSoup as bs

productID = '1'             #產品種類
dirName = 'Cosmetics_1'  #資料夾名稱
porExclude = '40849|54419|47002|668|24732|40609|44964|20136|42920'       #斷線當下那頁已抓完的產品編號,跳過這幾個編號不抓

headers = {
       'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36'   
    }
nextpage='https://www.urcosme.com/find-product/{}?page={}'                      #首頁 > 使用心得 - 找產品 > 臉部卸妝
domain='https://www.urcosme.com'

def getcontent():
    nextpage2 = '{}?page={}&review_search%5Bage%5D=%E4%B8%8D%E9%99%90&review_search%5Beffect%5D=%E4%B8%8D%E9%99%90&review_search%5Bscore%5D=%E4%B8%8D%E9%99%90&review_search%5Bskin%5D=%E4%B8%8D%E9%99%90' 
    filename = '{}/'.format(dirName) + name_modify + "/" + name_modify + '.csv' #檔名
    print filename
    f = open(filename, 'a+')         
    for i in range(1,page+1):    
        res = requests.get(nextpage2.format(nextpage1,i), headers=headers)     
        print nextpage2.format(nextpage1,i)                                    #nextpage2共2個參數:"要進入心得頁面的網址"與"頁數"
        soup = bs(res.text)     
        items = soup.select('.items')[0]
        count = 0
        for li in items:
            try:
                if li.select('.user-name'):                                        #進入每個產品的心得內容，如有user填寫才進入，廣告剔除               
                    count = count + 1
                    nextpage3 = domain + li.select('.img-anchor a')[0]['href']
                    #print nextpage3                
                    res2 = requests.get(nextpage3, headers=headers)
                    soup = bs(res2.text)                                           #下面就列印 內容了!!            
                    f.write(products_number + '|')                                                 #產品編號
                    f.write(li.select('.img-anchor a')[0]['href'].split('/reviews/')[1] + '|')     #心得編號
                    f.write(soup.select('.age')[0].text.strip().encode("utf-8") + '|')             #年齡
                    f.write(soup.select('.review-effects')[0].text.strip().encode("utf-8") + '|')  #效果
                    f.write(soup.select('.score')[0].text.strip().encode("utf-8") + '|')           #評分
                    f.write(soup.select('.skin')[0].text.strip().encode("utf-8") + '|')            #使用膚質
                    f.write(soup.select('.type')[0].text.strip().encode("utf-8") + '|')            #體驗方式
                    f.write(soup.select('.season')[0].text.strip().encode("utf-8") + '|')          #使用季節
                    f.write(soup.select('.enviroment')[0].text.strip().encode("utf-8") + '|')      #使用環境
                    f.write(soup.select('.skin')[0].text.strip().encode("utf-8"))                  #使用膚質               
                    #f.write(soup.select('.user-name a')[0]['href'].split('beauty_diary/')[1].strip().split('/reviews')[0].encode("utf-8"))#會員編號
                    f.write('\n') 
                    print '產品p', j , productName,'心得p',i,'No', count, nextpage3
            except IndexError:
                pass
                f.write(products_number + '|')                                                 #產品編號
                f.write(li.select('.img-anchor a')[0]['href'].split('/reviews/')[1] + '|')     #心得編號
                f.write('\n')
        time.sleep(3)
    f.close()

for j in range(48,100):                                                           #抓取頁面
    
    page_number = str(j)
    page_format = nextpage.format(productID,j)
    res = requests.get(page_format,headers=headers)
    soup = bs(res.text)
    item_info = soup.select('.item-info')                                       #抓取每個產品
   
    for li in item_info:                    
        products_number = li.select('a')[0]['href'].split('/products/')[1]      #產品編號
        if re.match(porExclude,products_number) is None:             #判斷這個產品編號抓過沒, 沒抓過才抓
            nextpage1 = domain + li.select('a')[0]['href']           #要進入心得頁面的網址(https://www.urcosme.com/products/63356)    
            print '產品list的 page=', j , nextpage1
            print nextpage1
            res1 = requests.get(nextpage1,headers=headers)
            soup = bs(res1.text)
            productName = soup.select('.val')[0].text.split('?')[0].strip()
            folderName = page_number+'_'+ productName +'_'+ products_number
            print folderName                                                       #建立資料夾的名稱
            name_modify = re.sub(r'>|/|<|＜|．|:|\?|\\','',folderName) 
            os.makedirs('{}/'.format(dirName) + name_modify)
            if len(soup.select('.pagination'))>0:                                  #如果"要進入心得頁面的網址"下面有頁數，
                pagination = soup.select('.pagination')[0]                         #則抓取頁數                           
                page = int(pagination.select('a')[-2].text)         
                #print page                                                        #頁數         
                getcontent()           
            else:                                                                  #如果"要進入心得頁面的網址"下面no頁數:設定1頁
                page = 1
                #print page           
                getcontent()
print "End^^~"

產品list的 page= 48 https://www.urcosme.com/products/13558
https://www.urcosme.com/products/13558
48_花語-精華卸妝乳_13558
Cosmetics_1/48_花語-精華卸妝乳_13558/48_花語-精華卸妝乳_13558.csv
https://www.urcosme.com/products/13558?page=1&review_search%5Bage%5D=%E4%B8%8D%E9%99%90&review_search%5Beffect%5D=%E4%B8%8D%E9%99%90&review_search%5Bscore%5D=%E4%B8%8D%E9%99%90&review_search%5Bskin%5D=%E4%B8%8D%E9%99%90
產品p 48 花語-精華卸妝乳 心得p 1 No 1 https://www.urcosme.com/reviews/356676
產品p 48 花語-精華卸妝乳 心得p 1 No 2 https://www.urcosme.com/reviews/313463
產品p 48 花語-精華卸妝乳 心得p 1 No 3 https://www.urcosme.com/reviews/241718
產品p 48 花語-精華卸妝乳 心得p 1 No 4 https://www.urcosme.com/reviews/230189
產品p 48 花語-精華卸妝乳 心得p 1 No 5 https://www.urcosme.com/reviews/187241
產品list的 page= 48 https://www.urcosme.com/products/47101
https://www.urcosme.com/products/47101
48_茶樹淨化深層卸妝棉_47101
Cosmetics_1/48_茶樹淨化深層卸妝棉_47101/48_茶樹淨化深層卸妝棉_47101.csv
https://www.urcosme.com/products/47101?page=1&review_search%5Bage%5D=%E4%B8%8D%E9%99%90&review_search%5Beffect%5D=%E4

C:\Python27\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


KeyboardInterrupt: 

In [2]:
#斷線後指定頁數用

import shutil
import os
import time
import sys
import requests
import re
from bs4 import BeautifulSoup as bs

productID = '11'               #產品種類
dirName = 'Cosmetics_11_17-20'  #資料夾名稱
products_number = '5323'        #產品編號

headers = {
       'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36'   
    }
nextpage='https://www.urcosme.com/find-product/{}?page={}'                      #首頁 > 使用心得 - 找產品 > 臉部卸妝
domain='https://www.urcosme.com'

def getcontent():
    nextpage2 = '{}?page={}&review_search%5Bage%5D=%E4%B8%8D%E9%99%90&review_search%5Beffect%5D=%E4%B8%8D%E9%99%90&review_search%5Bscore%5D=%E4%B8%8D%E9%99%90&review_search%5Bskin%5D=%E4%B8%8D%E9%99%90' 
    filename = '{}/'.format(dirName) + name_modify + "/" + name_modify + '.csv' #檔名
    print filename
    f = open(filename, 'a+')         
    for i in range(68,69):    ###############心得第幾頁################
        res = requests.get(nextpage2.format(nextpage1,i), headers=headers)     
        print nextpage2.format(nextpage1,i)                                 
        soup = bs(res.text)     
        items = soup.select('.items')[0]
        for li in items:
            if li.select('.user-name'):                                                      
                nextpage3 = domain + li.select('.img-anchor a')[0]['href']               
                res2 = requests.get(nextpage3, headers=headers)
                soup = bs(res2.text)                                                     
                f.write(products_number + '|')                                                 #產品編號
                f.write(li.select('.img-anchor a')[0]['href'].split('/reviews/')[1] + '|')     #心得編號
                f.write(soup.select('.age')[0].text.strip().encode("utf-8") + '|')             #年齡
                f.write(soup.select('.review-effects')[0].text.strip().encode("utf-8") + '|')  #效果
                f.write(soup.select('.score')[0].text.strip().encode("utf-8") + '|')           #評分
                f.write(soup.select('.skin')[0].text.strip().encode("utf-8") + '|')            #使用膚質
                f.write(soup.select('.type')[0].text.strip().encode("utf-8") + '|')            #體驗方式
                f.write(soup.select('.season')[0].text.strip().encode("utf-8") + '|')          #使用季節
                f.write(soup.select('.enviroment')[0].text.strip().encode("utf-8") + '|')      #使用環境
                f.write(soup.select('.skin')[0].text.strip().encode("utf-8"))                  #使用膚質               
                #f.write(soup.select('.user-name a')[0]['href'].split('beauty_diary/')[1].strip().split('/reviews')[0].encode("utf-8"))#會員編號
                f.write('\n')
                print '商品page',j,'心得page',i,nextpage3 
        time.sleep(10)
    f.close()

for j in range(17,20):         #########商品第幾頁#######
    print 'product page=', j                                                  
    page_number = str(j)
    page_format = nextpage.format(productID,j)
    res = requests.get(page_format,headers=headers)
    soup = bs(res.text)
    item_info = soup.select('.item-info')                                     
     
    for li in item_info:                    
        nextpage1 = 'https://www.urcosme.com/products/{}'.format(products_number) #要進入心得頁面的網址(https://www.urcosme.com/products/63356)    
        print nextpage1
        res1 = requests.get(nextpage1,headers=headers)
        soup = bs(res1.text)
        folderName = page_number+'_'+soup.select('.val')[0].text.split('?')[0].strip()+'_'+ products_number
        print folderName                                                       #建立資料夾的名稱
        name_modify = re.sub(r'>|/|<|＜|．|\?|\\','',folderName) 
        #os.makedirs('{}/'.format(dirName) + name_modify)
        if len(soup.select('.pagination'))>0:                                  #如果"要進入心得頁面的網址"下面有頁數，
            pagination = soup.select('.pagination')[0]                         #則抓取頁數                           
            page = int(pagination.select('a')[-2].text)         
            #print page                                                        #頁數         
            getcontent()           
        else:                                                                  #如果"要進入心得頁面的網址"下面no頁數:設定1頁
            page = 1
            #print page           
            getcontent()
print "End^^~"

product page= 17
https://www.urcosme.com/products/5323
17_細白修護面膜_5323
Cosmetics_11_17-20/17_細白修護面膜_5323/17_細白修護面膜_5323.csv
https://www.urcosme.com/products/5323?page=68&review_search%5Bage%5D=%E4%B8%8D%E9%99%90&review_search%5Beffect%5D=%E4%B8%8D%E9%99%90&review_search%5Bscore%5D=%E4%B8%8D%E9%99%90&review_search%5Bskin%5D=%E4%B8%8D%E9%99%90
商品page 17 心得page 68 https://www.urcosme.com/reviews/229
商品page 17 心得page 68 https://www.urcosme.com/reviews/197
商品page 17 心得page 68 https://www.urcosme.com/reviews/190
商品page 17 心得page 68 https://www.urcosme.com/reviews/188
商品page 17 心得page 68 https://www.urcosme.com/reviews/180
商品page 17 心得page 68 https://www.urcosme.com/reviews/166
商品page 17 心得page 68 https://www.urcosme.com/reviews/139
https://www.urcosme.com/products/5323
17_細白修護面膜_5323
Cosmetics_11_17-20/17_細白修護面膜_5323/17_細白修護面膜_5323.csv
https://www.urcosme.com/products/5323?page=68&review_search%5Bage%5D=%E4%B8%8D%E9%99%90&review_search%5Beffect%5D=%E4%B8%8D%E9%99%90&review_search%5Bscore%5D=%

IOError: [Errno 2] No such file or directory: u'Cosmetics_11_17-20/18_\u7d30\u767d\u4fee\u8b77\u9762\u819c_5323/18_\u7d30\u767d\u4fee\u8b77\u9762\u819c_5323.csv'

In [ ]:
import re
str = '123|223'
if re.match('1234',str) is None:
    print 'not match'
else:
    print 'match'